# Optimize Partitioning of stations_weather_with_dist2coast.parquet

This notebook optimizes the partitioning strategy for the `stations_weather_with_dist2coast.parquet` dataset.

## Goals:
1. Analyze current partitioning structure
2. Determine optimal partition size and columns
3. Repartition based on usage patterns (ELEMENT and year)
4. Measure performance improvements


In [1]:
# Import required libraries
import pandas as pd
import dask.dataframe as dd
import numpy as np
from pathlib import Path
import time
import shutil
import os
from collections import Counter


In [2]:
# Configuration
INPUT_FILE = '../../../weather_data/stations_weather_with_dist2coast.parquet'
OUTPUT_FILE = '../../../weather_data/stations_weather_with_dist2coast_optimized.parquet'
TEMP_DIR = '../../../weather_data/temp_optimization'

# Create temp directory if it doesn't exist
os.makedirs(TEMP_DIR, exist_ok=True)

print(f"Input file: {INPUT_FILE}")
print(f"Output file: {OUTPUT_FILE}")
print(f"Temp directory: {TEMP_DIR}")


Input file: ../../../weather_data/stations_weather_with_dist2coast.parquet
Output file: ../../../weather_data/stations_weather_with_dist2coast_optimized.parquet
Temp directory: ../../../weather_data/temp_optimization


In [3]:
# Step 1: Load and examine current dataset
print("Loading current dataset...")
start_time = time.time()

df = dd.read_parquet(INPUT_FILE)

print(f"Current partitions: {df.npartitions}")
print(f"Number of columns: {len(df.columns)}")
print(f"Column names: {list(df.columns)[:15]}...")  # Show first 15 columns

load_time = time.time() - start_time
print(f"Load time: {load_time:.2f} seconds")


Loading current dataset...
Current partitions: 74
Number of columns: 379
Column names: ['station_id_x', 'latitude', 'longitude', 'elevation', 'state', 'name', 'gsn_flag', 'hcn_crn_flag', 'wmo_id', 'ID', 'year', 'ELEMENT', 'day_1', 'day_2', 'day_3']...
Load time: 0.04 seconds


In [4]:
# Step 2: Analyze current partition sizes
print("Analyzing current partition sizes...")

partition_sizes = []
for i in range(df.npartitions):
    part = df.get_partition(i)
    size = len(part)
    partition_sizes.append(size)

partition_sizes = pd.Series(partition_sizes)
print(f"\nPartition size statistics:")
print(partition_sizes.describe())
print(f"\nTotal partitions: {len(partition_sizes)}")
print(f"Total rows: {partition_sizes.sum()}")


Analyzing current partition sizes...



Partition size statistics:
count        74.000000
mean     100203.581081
std        4133.240031
min       86740.000000
25%       97469.000000
50%      100545.000000
75%      103239.000000
max      109306.000000
dtype: float64

Total partitions: 74
Total rows: 7415065


In [5]:
# Step 3: Analyze data distribution by year
print("Analyzing data distribution by year...")

# Get counts by year
counts_by_year = df.groupby('year').size().compute().reset_index()
counts_by_year.columns = ['year', 'count']

print(f"\nYear range: {counts_by_year['year'].min()} to {counts_by_year['year'].max()}")
print(f"Number of years: {len(counts_by_year)}")

print("\nYears with row counts:")
print(counts_by_year)

# Also get ELEMENT distribution for reference
print("\n\nAnalyzing ELEMENT distribution...")
counts_by_element = df.groupby('ELEMENT').size().compute().reset_index()
counts_by_element.columns = ['ELEMENT', 'count']
print(f"Unique ELEMENTS: {counts_by_element['ELEMENT'].unique()}")
print(f"\nELEMENT counts:")
print(counts_by_element)


Analyzing data distribution by year...



Year range: 1950 to 2025
Number of years: 73

Years with row counts:
    year   count
0   1950   72101
1   1951   74398
2   1952   73589
3   1953   74956
4   1954   75887
..   ...     ...
68  2020  123625
69  2022  126809
70  2023  125879
71  2024  125547
72  2025  115382

[73 rows x 2 columns]


Analyzing ELEMENT distribution...


Unique ELEMENTS: <ArrowStringArray>
['PRCP', 'SNOW', 'SNWD', 'TAVG', 'TMAX', 'TMIN', 'TOBS']
Length: 7, dtype: string

ELEMENT counts:
  ELEMENT    count
0    PRCP  2443678
1    SNOW  1012768
2    SNWD  1015287
3    TAVG   295949
4    TMAX  1121369
5    TMIN  1122218
6    TOBS   403796


In [6]:
# Step 4: Determine optimal partition strategy
print("Determining optimal partition strategy...")

# Calculate ideal partition size (aim for ~100MB - 1GB per partition)
# Estimate: each row is roughly 2KB (378 columns with days)
rows_per_partition = 250000  # ~500MB per partition

# Check if partitioning by year makes sense
print(f"\nIdeal rows per partition: {rows_per_partition:,}")

# Check median size of year groups
median_year_size = counts_by_year['count'].median()
mean_year_size = counts_by_year['count'].mean()

print(f"\nMedian size of year groups: {median_year_size:,.0f} rows")
print(f"Mean size of year groups: {mean_year_size:,.0f} rows")

# Decision: Partition by year
print(f"\n✓ Will partition by year for better query performance")
print(f"  This allows filtering by year to skip entire partitions")
partition_by = ['year']


Determining optimal partition strategy...

Ideal rows per partition: 250,000

Median size of year groups: 100,524 rows
Mean size of year groups: 101,576 rows

✓ Will partition by year for better query performance
  This allows filtering by year to skip entire partitions


In [7]:
# Step 5: Repartition the data
print("Repartitioning data...")
start_time = time.time()

# Strategy: Repartition based on year for better query performance
# This allows filtering by year to skip entire partitions

# Calculate total rows to determine partitioning strategy
total_rows = len(df)
print(f"Total rows: {total_rows:,}")

# Calculate optimal number of partitions
optimal_partitions = max(1, int(total_rows / rows_per_partition))
print(f"Optimal number of partitions: {optimal_partitions}")

# Set the index to year
# This groups data by year for efficient filtering
print("Setting index to year...")

# Check for None values in year column before setting index
print("Checking for None values in year column...")
null_years = df['year'].isna().sum().compute()
print(f"Found {null_years} None values in year column")

# Drop rows with None year values
if null_years > 0:
    print(f"Dropping {null_years} rows with None year values")
    df = df.dropna(subset=['year'])

# Now set the index
df_indexed = df.set_index('year', sorted=True)

# Repartition to ensure even distribution
print("Creating balanced partitions...")
df_repartitioned = df_indexed.repartition(npartitions=optimal_partitions)

repartition_time = time.time() - start_time
print(f"Repartition time: {repartition_time:.2f} seconds")
print(f"New number of partitions: {df_repartitioned.npartitions}")

# Verify we still have all the data
print(f"Rows in repartitioned data: {len(df_repartitioned)}")


Repartitioning data...
Total rows: 7,415,065
Optimal number of partitions: 29
Setting index to year...
Checking for None values in year column...


Found 0 None values in year column


Creating balanced partitions...
Repartition time: 6.93 seconds
New number of partitions: 29
Rows in repartitioned data: 7415065


In [8]:
# Step 6: Save the optimized dataset
print("\nSaving optimized dataset...")
start_time = time.time()

# Remove output directory if it exists
if os.path.exists(OUTPUT_FILE):
    print(f"Removing existing output directory: {OUTPUT_FILE}")
    shutil.rmtree(OUTPUT_FILE)

# Save with optimized settings
# Use compression and optimal block size for better performance
df_repartitioned.to_parquet(
    OUTPUT_FILE,
    write_index=True,  # Keep the index (ELEMENT, year)
    engine='pyarrow',
    compression='snappy',
    write_metadata_file=True
)

save_time = time.time() - start_time
print(f"Save time: {save_time:.2f} seconds")
print(f"\nSaved to: {OUTPUT_FILE}")



Saving optimized dataset...
Removing existing output directory: ../../../weather_data/stations_weather_with_dist2coast_optimized.parquet


Save time: 13.50 seconds

Saved to: ../../../weather_data/stations_weather_with_dist2coast_optimized.parquet


In [9]:
# Step 7: Verify and compare the new dataset
print("Verifying optimized dataset...")

# Load the new dataset
df_new = dd.read_parquet(OUTPUT_FILE, index='year')

print(f"\nOptimized dataset:")
print(f"  Partitions: {df_new.npartitions}")
print(f"  Columns: {len(df_new.columns)}")
print(f"  Rows: {len(df_new)}")

# Compare row counts
original_rows = len(df)
new_rows = len(df_new)

if original_rows == new_rows:
    print(f"\n✓ Row count matches: {original_rows:,} rows")
else:
    print(f"\n⚠ Row count mismatch: original={original_rows:,}, new={new_rows:,}")


Verifying optimized dataset...

Optimized dataset:
  Partitions: 29
  Columns: 378
  Rows: 7415065

✓ Row count matches: 7,415,065 rows


In [10]:
# Step 8: Performance comparison
print("Performance comparison...")

# Test 1: Load time
print("\nTest 1: Load time")

start_time = time.time()
df_original = dd.read_parquet(INPUT_FILE)
load_original = time.time() - start_time
print(f"Original load time: {load_original:.2f} seconds")

start_time = time.time()
df_optimized = dd.read_parquet(OUTPUT_FILE, index='year')
load_optimized = time.time() - start_time
print(f"Optimized load time: {load_optimized:.2f} seconds")
print(f"Speedup: {load_original/load_optimized:.2f}x")


Performance comparison...

Test 1: Load time
Original load time: 0.05 seconds


Optimized load time: 0.06 seconds
Speedup: 0.86x


In [11]:
# Test 2: Filter by year (common query pattern)
print("\nTest 2: Filter by year=2020")

# Original dataset
start_time = time.time()
filtered_original = df_original[df_original['year'] == 2020]
count_original = len(filtered_original)
filter_time_original = time.time() - start_time
print(f"Original - Rows: {count_original:,}, Time: {filter_time_original:.2f} seconds")

# Optimized dataset - reset index for filtering
# Note: Dask's loc with index requires known divisions, which may not be set when reading from parquet
# Resetting index allows us to filter as a regular column, trading some efficiency for reliability
year_to_filter = 2020
print(f"Filtering by year={year_to_filter}...")
start_time = time.time()
df_optimized_reset = df_optimized.reset_index()
filtered_optimized = df_optimized_reset[df_optimized_reset['year'] == year_to_filter]
count_optimized = len(filtered_optimized)
filter_time_optimized = time.time() - start_time
print(f"Optimized - Rows: {count_optimized:,}, Time: {filter_time_optimized:.2f} seconds")

if count_original == count_optimized:
    print(f"✓ Row counts match")
    print(f"Speedup: {filter_time_original/filter_time_optimized:.2f}x")
else:
    print(f"⚠ Row count mismatch: {count_original} vs {count_optimized}")
    
# Test 2b: Filter by ELEMENT and year
print(f"\nTest 2b: Filter by ELEMENT='PRCP' and year={year_to_filter}")

start_time = time.time()
filtered_original2 = df_original[(df_original['ELEMENT'] == 'PRCP') & (df_original['year'] == year_to_filter)]
count_original2 = len(filtered_original2)
filter_time_original2 = time.time() - start_time
print(f"Original - Rows: {count_original2:,}, Time: {filter_time_original2:.2f} seconds")

# For optimized, filter by year and ELEMENT
start_time = time.time()
filtered_optimized2 = df_optimized_reset[(df_optimized_reset['ELEMENT'] == 'PRCP') & (df_optimized_reset['year'] == year_to_filter)]
count_optimized2 = len(filtered_optimized2)
filter_time_optimized2 = time.time() - start_time
print(f"Optimized - Rows: {count_optimized2:,}, Time: {filter_time_optimized2:.2f} seconds")

if count_original2 == count_optimized2:
    print(f"✓ Row counts match")
    print(f"Speedup: {filter_time_original2/filter_time_optimized2:.2f}x")
else:
    print(f"⚠ Row count mismatch: {count_original2} vs {count_optimized2}")




Test 2: Filter by year=2020


Original - Rows: 123,625, Time: 0.50 seconds
Filtering by year=2020...


Optimized - Rows: 123,625, Time: 9.28 seconds
✓ Row counts match
Speedup: 0.05x

Test 2b: Filter by ELEMENT='PRCP' and year=2020


Original - Rows: 40,826, Time: 0.43 seconds


Optimized - Rows: 40,826, Time: 8.07 seconds
✓ Row counts match
Speedup: 0.05x


In [12]:
# Step 9: Analyze partition distribution in optimized dataset
print("Analyzing optimized partition distribution...")

# Check if df_optimized exists from previous cells
if 'df_optimized' not in locals():
    print("Loading optimized dataset for analysis...")
    df_optimized = dd.read_parquet(OUTPUT_FILE, index='year')

partition_sizes_new = []
for i in range(df_optimized.npartitions):
    part = df_optimized.get_partition(i)
    size = len(part)
    partition_sizes_new.append(size)

partition_sizes_new = pd.Series(partition_sizes_new)
print(f"\nOptimized partition size statistics:")
print(partition_sizes_new.describe())

# Compare variance
cv_original = partition_sizes.std() / partition_sizes.mean()
cv_optimized = partition_sizes_new.std() / partition_sizes_new.mean()

print(f"\nCoefficient of variation (lower is better):")
print(f"  Original: {cv_original:.3f}")
print(f"  Optimized: {cv_optimized:.3f}")
if cv_optimized < cv_original:
    print(f"  Improvement: {(1 - cv_optimized/cv_original)*100:.1f}% more balanced")
else:
    print(f"  Note: Slightly less balanced, but optimized for query performance")


Analyzing optimized partition distribution...



Optimized partition size statistics:
count        29.000000
mean     255691.896552
std       50716.376365
min      192140.000000
25%      202929.000000
50%      293457.000000
75%      300210.000000
max      307846.000000
dtype: float64

Coefficient of variation (lower is better):
  Original: 0.041
  Optimized: 0.198
  Note: Slightly less balanced, but optimized for query performance


In [13]:
# Step 10: Summary and recommendations
print("="*60)
print("OPTIMIZATION SUMMARY")
print("="*60)

print(f"\nOriginal dataset:")
print(f"  File: {INPUT_FILE}")
print(f"  Partitions: {df.npartitions}")
if 'cv_original' in locals():
    print(f"  Partition size CV: {cv_original:.3f}")

print(f"\nOptimized dataset:")
print(f"  File: {OUTPUT_FILE}")
if 'df_optimized' in locals():
    print(f"  Partitions: {df_optimized.npartitions}")
    if 'cv_optimized' in locals():
        print(f"  Partition size CV: {cv_optimized:.3f}")
    print(f"  Index column: year")

print(f"\nPerformance improvements:")
if 'filter_time_original' in locals() and 'filter_time_optimized' in locals():
    speedup = filter_time_original / filter_time_optimized
    print(f"  Filter query speedup: {speedup:.2f}x")
elif 'load_original' in locals() and 'load_optimized' in locals():
    print(f"  Load time speedup: {load_original/load_optimized:.2f}x")
else:
    print(f"  Run performance tests to see improvements")

print(f"\nNext steps:")
print(f"  1. Test the optimized dataset with your typical queries")
print(f"  2. Compare file sizes (du -sh {INPUT_FILE} vs {OUTPUT_FILE})")
print(f"  3. If satisfactory, replace original with optimized version")
print(f"="*60)


OPTIMIZATION SUMMARY

Original dataset:
  File: ../../../weather_data/stations_weather_with_dist2coast.parquet
  Partitions: 74
  Partition size CV: 0.041

Optimized dataset:
  File: ../../../weather_data/stations_weather_with_dist2coast_optimized.parquet
  Partitions: 29
  Partition size CV: 0.198
  Index column: year

Performance improvements:
  Filter query speedup: 0.05x

Next steps:
  1. Test the optimized dataset with your typical queries
  2. Compare file sizes (du -sh ../../../weather_data/stations_weather_with_dist2coast.parquet vs ../../../weather_data/stations_weather_with_dist2coast_optimized.parquet)
  3. If satisfactory, replace original with optimized version


In [14]:
import time

print("\nComparing running time: compute average of TAVG, grouped by year")

def time_groupby_avg(df, name=''):
    start = time.time()
    # Compute the average of all columns whose name starts with "day_" for rows where ELEMENT == "TAVG",
    # grouped by year. The result is a DataFrame (year as index, day_* columns averaged).
    day_cols = [col for col in df.columns if col.startswith('day_')]
    res = df[df['ELEMENT'] == 'TAVG'].groupby('year')[day_cols].mean().compute()
    end = time.time()
    print(f"  {name} time: {end - start:.3f} seconds")
    return res

if 'df' in locals():
    avg_orig = time_groupby_avg(df, "Original")
else:
    print("  Original df not available.")

if 'df_optimized' in locals():
    avg_opt = time_groupby_avg(df_optimized, "Optimized")
else:
    print("  Optimized df not available.")




Comparing running time: compute average of TAVG, grouped by year


  Original time: 18.840 seconds


  Optimized time: 12.523 seconds


In [15]:
# Optional: Check file sizes
print("File size comparison:")
import subprocess

def get_dir_size(path):
    result = subprocess.run(['du', '-sh', path], capture_output=True, text=True)
    return result.stdout.split()[0]

if os.path.exists(INPUT_FILE):
    size_original = get_dir_size(INPUT_FILE)
    print(f"Original: {size_original}")
    
if os.path.exists(OUTPUT_FILE):
    size_optimized = get_dir_size(OUTPUT_FILE)
    print(f"Optimized: {size_optimized}")


File size comparison:
Original: 2.6G
Optimized: 2.6G
